In [1]:
import requests
from bs4 import BeautifulSoup
import polars as pl
import json
from tqdm import tqdm
import time

# URL сайта, который вы хотите парсить
url = 'https://russiabase.ru/prices?brand=292'  # Замените на нужный вам URL

# Отправляем GET-запрос к сайту
response = requests.get(url)

# Проверяем, успешен ли запрос
if response.status_code == 200:
    # Парсим содержимое страницы
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Ключевые слова для фильтрации
    keywords = ["@context"]
    
    # Находим все div и фильтруем по ключевым словам
    divs = soup.find_all('script')
    
    # Список для хранения строк JSON
    json_strings = []

    for div in divs:
        # Проверяем, содержит ли текст div хотя бы одно из ключевых слов
        if any(keyword in div.text for keyword in keywords):
            # Добавляем текст в формате JSON в список
            json_strings.append(div.text.strip())

    # Список для хранения словарей
    data = []

    # Обрабатываем каждую строку JSON
    for json_string in json_strings:
        try:
            data_dict = json.loads(json_string)  # Преобразуем строку JSON в словарь
            data.append(data_dict)  # Добавляем словарь в список
        except json.JSONDecodeError:
            print(f"Ошибка декодирования JSON: {json_string}")

    # Создаем DataFrame из списка словарей
    df = pl.DataFrame(data)

    # Разбиваем поле 'description' на отдельные записи
    expanded_rows = []

    for row in df.iter_rows(named=True):
        description = row['description']
        # Разбиваем строку по разделителю ';' и убираем лишние пробелы
        items = [item.strip() for item in description.split(';') if item.strip()]
        for item in items:
            # Разбиваем элемент на две части по разделителю '-' (или любому другому)
            parts = item.split(' - ')
            description_part1 = parts[0].strip() if len(parts) > 0 else ''
            description_part2 = parts[1].strip() if len(parts) > 1 else ''
            
            # Создаем новую запись с оригинальными данными и разбитыми элементами
            expanded_rows.append({
                '@context': row['@context'],
                '@type': row['@type'],
                'legalName': row['legalName'],
                'address': row['address'],
                'description_part1': description_part1,
                'description_part2': description_part2
            })
    
    # Создаем новый DataFrame из разбитых записей
    expanded_df = pl.DataFrame(expanded_rows)

    expanded_df = expanded_df.rename({"description_part1": "Название_топлива", "description_part2": "Цена_топлива"})

    # Выводим новый DataFrame
    print(expanded_df)
else:
    print(f'Ошибка при запросе: {response.status_code}')


shape: (89, 6)
┌────────────────────┬──────────────┬──────────────────┬─────────────────────────────┬──────────────────┬──────────────┐
│ @context           ┆ @type        ┆ legalName        ┆ address                     ┆ Название_топлива ┆ Цена_топлива │
│ ---                ┆ ---          ┆ ---              ┆ ---                         ┆ ---              ┆ ---          │
│ str                ┆ str          ┆ str              ┆ str                         ┆ str              ┆ str          │
╞════════════════════╪══════════════╪══════════════════╪═════════════════════════════╪══════════════════╪══════════════╡
│ https://schema.org ┆ Organization ┆ Роснефть РН-Карт ┆ Придорожная ул., 1, Казань, ┆ Аи-92            ┆ 54.70        │
│                    ┆              ┆                  ┆ Ре…                         ┆                  ┆              │
│ https://schema.org ┆ Organization ┆ Роснефть РН-Карт ┆ Придорожная ул., 1, Казань, ┆ Аи-95            ┆ 58.80        │
│                

In [2]:
list_azs = ['https://russiabase.ru/prices?brand=292'] + [f'https://russiabase.ru/prices?brand=292&page={i}' for i in range(2, 24)]
len(list_azs)

23

In [7]:
import requests
from bs4 import BeautifulSoup
import polars as pl
import json
from tqdm import tqdm
import time

expanded_df_base = None

for i, elem in tqdm(enumerate(list_azs)):
    print(elem)
    url = f'{elem}'
    
    # Отправляем GET-запрос к сайту
    response = requests.get(url)
    
    # Проверяем, успешен ли запрос
    if response.status_code == 200:
        # Парсим содержимое страницы
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Ключевые слова для фильтрации
        keywords = ["@context"]
        
        # Находим все div и фильтруем по ключевым словам
        divs = soup.find_all('script')
        
        # Список для хранения строк JSON
        json_strings = []
    
        for div in divs:
            # Проверяем, содержит ли текст div хотя бы одно из ключевых слов
            if any(keyword in div.text for keyword in keywords):
                # Добавляем текст в формате JSON в список
                json_strings.append(div.text.strip())
    
        # Список для хранения словарей
        data = []
    
        # Обрабатываем каждую строку JSON
        for json_string in json_strings:
            try:
                data_dict = json.loads(json_string)  # Преобразуем строку JSON в словарь
                data.append(data_dict)  # Добавляем словарь в список
            except json.JSONDecodeError:
                print(f"Ошибка декодирования JSON: {json_string}")
    
        # Создаем DataFrame из списка словарей
        df = pl.DataFrame(data)
    
        # Разбиваем поле 'description' на отдельные записи
        expanded_rows = []
    
        for row in df.iter_rows(named=True):
            description = row['description']
            # Разбиваем строку по разделителю ';' и убираем лишние пробелы
            items = [item.strip() for item in description.split(';') if item.strip()]
            for item in items:
                # Разбиваем элемент на две части по разделителю '-' (или любому другому)
                parts = item.split(' - ')
                description_part1 = parts[0].strip() if len(parts) > 0 else ''
                description_part2 = parts[1].strip() if len(parts) > 1 else ''
                
                # Создаем новую запись с оригинальными данными и разбитыми элементами
                expanded_rows.append({
                    '@context': row['@context'],
                    '@type': row['@type'],
                    'legalName': row['legalName'],
                    'address': row['address'],
                    'description_part1': description_part1,
                    'description_part2': description_part2
                })
        
        # Создаем новый DataFrame из разбитых записей
        expanded_df = pl.DataFrame(expanded_rows)
    
        expanded_df = expanded_df.rename({"description_part1": "Название_топлива", "description_part2": "Цена_топлива"})
    
        # Выводим новый DataFrame

    if expanded_df_base is None:
        expanded_df_base = expanded_df
    else:
        expanded_df_base = pl.concat([expanded_df_base, expanded_df], how='vertical')

    time.sleep(1)

0it [00:00, ?it/s]

https://russiabase.ru/prices?brand=119
shape: (94, 6)
┌────────────────────┬──────────────┬────────────────────────┬───────────────────────┬──────────────────┬──────────────┐
│ @context           ┆ @type        ┆ legalName              ┆ address               ┆ Название_топлива ┆ Цена_топлива │
│ ---                ┆ ---          ┆ ---                    ┆ ---                   ┆ ---              ┆ ---          │
│ str                ┆ str          ┆ str                    ┆ str                   ┆ str              ┆ str          │
╞════════════════════╪══════════════╪════════════════════════╪═══════════════════════╪══════════════════╪══════════════╡
│ https://schema.org ┆ Organization ┆ ООО                    ┆ Самарская область,    ┆ Аи-92            ┆ 56.49        │
│                    ┆              ┆ "ЛУКОЙЛ-Интер-Кард"    ┆ Волжский р-…          ┆                  ┆              │
│ https://schema.org ┆ Organization ┆ ООО                    ┆ Самарская область,    ┆ Аи-95+      

1it [00:01,  1.87s/it]

https://russiabase.ru/prices?brand=119&page=2
shape: (79, 6)
┌────────────────────┬──────────────┬───────────────────────┬────────────────────────┬──────────────────┬──────────────┐
│ @context           ┆ @type        ┆ legalName             ┆ address                ┆ Название_топлива ┆ Цена_топлива │
│ ---                ┆ ---          ┆ ---                   ┆ ---                    ┆ ---              ┆ ---          │
│ str                ┆ str          ┆ str                   ┆ str                    ┆ str              ┆ str          │
╞════════════════════╪══════════════╪═══════════════════════╪════════════════════════╪══════════════════╪══════════════╡
│ https://schema.org ┆ Organization ┆ Teboil №11236         ┆ Калининградская        ┆ Аи-92            ┆ 60.94        │
│                    ┆              ┆                       ┆ область, Зелен…        ┆                  ┆              │
│ https://schema.org ┆ Organization ┆ Teboil №11236         ┆ Калининградская        ┆ Аи-95

2it [00:03,  1.84s/it]

...

95it [03:17,  2.44s/it]

https://russiabase.ru/prices?brand=119&page=96
shape: (84, 6)
┌────────────────────┬──────────────┬────────────────────────┬───────────────────────┬──────────────────┬──────────────┐
│ @context           ┆ @type        ┆ legalName              ┆ address               ┆ Название_топлива ┆ Цена_топлива │
│ ---                ┆ ---          ┆ ---                    ┆ ---                   ┆ ---              ┆ ---          │
│ str                ┆ str          ┆ str                    ┆ str                   ┆ str              ┆ str          │
╞════════════════════╪══════════════╪════════════════════════╪═══════════════════════╪══════════════════╪══════════════╡
│ https://schema.org ┆ Organization ┆ ООО                    ┆ А216, 52, Советск,    ┆ Аи-92            ┆ 60.81        │
│                    ┆              ┆ "ЛУКОЙЛ-Интер-Кард"    ┆ Калининград…          ┆                  ┆              │
│ https://schema.org ┆ Organization ┆ ООО                    ┆ А216, 52, Советск,    ┆ Аи-9

96it [03:19,  2.29s/it]

https://russiabase.ru/prices?brand=119&page=97
shape: (75, 6)
┌────────────────────┬──────────────┬────────────────────────┬───────────────────────┬──────────────────┬──────────────┐
│ @context           ┆ @type        ┆ legalName              ┆ address               ┆ Название_топлива ┆ Цена_топлива │
│ ---                ┆ ---          ┆ ---                    ┆ ---                   ┆ ---              ┆ ---          │
│ str                ┆ str          ┆ str                    ┆ str                   ┆ str              ┆ str          │
╞════════════════════╪══════════════╪════════════════════════╪═══════════════════════╪══════════════════╪══════════════╡
│ https://schema.org ┆ Organization ┆ ООО                    ┆ пер. Красный,         ┆ Аи-92            ┆ 56.50        │
│                    ┆              ┆ "ЛУКОЙЛ-Интер-Кард"    ┆ Якшур-Бодья, рес…     ┆                  ┆              │
│ https://schema.org ┆ Organization ┆ ООО                    ┆ пер. Красный,         ┆ Аи-9

97it [03:21,  2.08s/it]


In [11]:
expanded_df_base.drop(['@context', '@type']).with_columns(pl.col('Цена_топлива').cast(pl.Float64).alias('Цена_топлива')).write_excel('башнефть.xlsx')